# Motion Prediction Baseline — Inference with PyTorch & L5Kit

_This notebook loads a trained BEV-based model and generates future trajectories for each agent in the test set._

## Notes

- You need a trained model checkpoint (e.g. `model_state_last.pth`) saved from the training notebook.
- A GPU is recommended for faster inference.
- Parts of code from the [official example](https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_prediction.ipynb)
- #### Version #1
- Single mode baseline (resnet-18)
- Trained for 25000 iterations (batch 32)
- Input size 300px, history 1s (10 frames)
- Adam (1e-3)
- MSE Loss

## Imports & Setup

We import PyTorch, L5Kit, and utility libraries.  Paths for the dataset and where to save the submission are defined here.

In [ ]:
import numpy as np
import os
import torch

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18
from tqdm import tqdm
from typing import Dict

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.evaluation import write_pred_csv
from l5kit.rasterization import build_rasterizer

In [ ]:
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

SINGLE_MODE_SUBMISSION = f"{DIR_INPUT}/single_mode_sample_submission.csv"
MULTI_MODE_SUBMISSION = f"{DIR_INPUT}/multi_mode_sample_submission.csv"

# Training notebook's output.
WEIGHT_FILE = "/kaggle/input/pytorch-baseline-train/model_state_last.pth"

## Configuration (cfg)

- `model_params`: same as in training (history and future frames).
- `raster_params`: raster size, pixel size, ego centre, map type.
- `val_data_loader` / `test_data_loader`: specify zarr keys (`scenes/validate.zarr` or `scenes/sample.zarr`), batch size and workers.
- `inference_params`: number of test scenes to process (can be limited for debug).

This dict mirrors the training config but points to the test split.

In [ ]:
cfg = {
    'format_version': 4,

    'model_params': {
        'model_architecture': 'resnet18',   
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
    },

    'raster_params': {
        'raster_size': [224, 224],          
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
        'disable_traffic_light_faces': False 
    },

    'test_data_loader': {
        'key': 'scenes/test.zarr', 
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 4,
    }
}

## Data Root & LocalDataManager

Set `L5KIT_DATA_FOLDER` and create a `LocalDataManager` so the loader can find Zarr files.

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

## Load Test Dataset & DataLoader

We build a rasteriser, open the test zarr, and create an `AgentDataset` or `EgoDataset` for inference.  The DataLoader wraps it with the chosen batch size and worker count.

Check `print(test_dataset)` to verify scene counts.

In [ ]:
# ===== INIT DATASET
test_cfg = cfg["test_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataloader)

## Model

Same architecture as training (e.g., ResNet-18).  
First conv is widened for map + history channels; the head outputs **`2 × future_num_frames`** per sample.

In [ ]:
class LyftModel(nn.Module):
    
    def __init__(self, cfg: Dict):
        super().__init__()
        
        self.backbone = resnet18(pretrained=False)
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 512

        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.logit = nn.Linear(4096, out_features=num_targets)
        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.head(x)
        x = self.logit(x)
        
        return x


## Initialise & Load Weights

- Instantiate `LyftModel(cfg)` and move to device.
- Load the trained state dict from the training notebook’s output.
- Set `model.eval()` and disable grads for inference.

In [ ]:
# ==== INIT MODEL (inference) ====
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = LyftModel(cfg).to(device)

if WEIGHT_FILE is not None:
    state = torch.load(WEIGHT_FILE, map_location=device)
    if isinstance(state, dict) and 'model_state_dict' in state:
        model.load_state_dict(state['model_state_dict'])
    else:
        model.load_state_dict(state)
else:
    raise FileNotFoundError("WEIGHT_FILE is None; attach your training notebook output or dataset and set WEIGHT_FILE path.")

model.eval()
torch.set_grad_enabled(False)

In [ ]:
device

## Predicting

Loop:
1) Send batch `image` to device.  
2) `outputs = model(inputs).reshape(targets.shape)` to get `(N, T, 2)`.  
3) Accumulate `outputs`, `timestamp`, and `track_id` for the submission.

In [ ]:
model.eval()

future_coords_offsets_pd = []
timestamps = []
agent_ids = []

with torch.no_grad():
    dataiter = tqdm(test_dataloader)
    
    for data in dataiter:

        inputs = data["image"].to(device)
        target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
        targets = data["target_positions"].to(device)

        outputs = model(inputs).reshape(targets.shape)
        
        future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())


## Save Submission

Concatenate per-batch arrays and write a Kaggle-ready CSV:
- `timestamps`
- `track_ids`
- `coords` (future `(x, y)` per step)

Verify the file path is `/kaggle/working/submission.csv` for Kaggle submissions.

In [ ]:
write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd))